In [1]:
import numpy as np
import matplotlib.pyplot as plt
import logging
import os
from pathlib import Path
from tqdm import trange
from flygym.mujoco import Parameters
from gymnasium.utils.env_checker import check_env
from flygym.mujoco.examples.turning_controller import HybridTurningNMF
from flygym.mujoco.arena import OdorArena

# Odor source: array of shape (num_odor_sources, 3) - xyz coords of odor sources
odor_source = np.array(
    [[24, 0, 1.5], [8, -4, 1.5], [16, 4, 1.5], [12, -8, 1.5], [20, 3, 1.5]]
)

# Peak intensities: array of shape (num_odor_sources, odor_dimesions)
# For each odor source, if the intensity is (x, 0) then the odor is in the 1st dimension
# (in this case yeast). If it's (0, x) then it's in the 2nd dimension (in this case
# sucrose)
peak_intensity = np.array([[1, 0], [2, 0], [3, 0], [4, 0], [5, 0]])

# Marker colors: array of shape (num_odor_sources, 4) - RGBA values for each marker,
# normalized to [0, 1]
# marker_colors = [[255, 127, 14], [31, 119, 180], [31, 119, 180]]
# marker_colors = np.array([[*np.array(color) / 255, 1] for color in marker_colors])

odor_dimesions = len(peak_intensity[0])

odor_valence = [10, 15, 20, 17, 6]

logging.basicConfig(level=logging.INFO)

In [2]:
# Set up the arena
arena = OdorArena(
    odor_source=odor_source,
    size=(50, 50),
    peak_intensity=peak_intensity,
    odor_valence=odor_valence,
    diffuse_func=lambda x: x**-2,
    marker_size=0.3,
)

(-0.5, 799.5, 607.5, -0.5)

In [3]:
# Add the NeuroMechFLy

contact_sensor_placements = [
    f"{leg}{segment}"
    for leg in ["LF", "LM", "LH", "RF", "RM", "RH"]
    for segment in ["Tibia", "Tarsus1", "Tarsus2", "Tarsus3", "Tarsus4", "Tarsus5"]
]
sim_params = Parameters(
    timestep=1e-4,
    render_mode="saved",
    render_playspeed=0.5,
    render_window_size=(800, 608),
    enable_olfaction=True,
    enable_adhesion=True,
    draw_adhesion=False,
    render_camera="birdeye_cam",
)
sim = HybridTurningNMF(
    sim_params=sim_params,
    arena=arena,
    spawn_pos=(0, 0, 0.2),
    contact_sensor_placements=contact_sensor_placements,
    simulation_time=10,
    food_loss_rate=0.00003,
    food_stocked_init=1.0,
    mating_state="mated",
    odor_score_reach_addition=2,
    odor_score_time_loss=1
)
for i in range(1):
    sim.step(np.zeros(2))
    sim.render()
fig, ax = plt.subplots(1, 1, figsize=(5, 4), tight_layout=True)
ax.imshow(sim._frames[-1])
ax.axis("off")

In [ ]:
decision_interval = 0.01
run_time = sim.simulation_time
num_decision_steps = int(run_time / decision_interval)
logging.info("Number of decision steps: ", num_decision_steps)
physics_steps_per_decision_step = int(decision_interval / sim_params.timestep)
logging.info("Physics steps per decision step: ", physics_steps_per_decision_step)

obs_hist = []
odor_history = []
obs, _ = sim.reset()

for i in trange(int(run_time / sim.sim_params.timestep)):
    # implement function to determine where to go next
    # food_source = 
    food_source = 0
    attractive_gain = -500
    control_signal = arena.generate_specific_turning_control(
        food_source, sim, attractive_gain
    )
    obs, _, _, _, _ = sim.step(control_signal)
    rendered_img = sim.render()
    obs_hist.append(obs)

print(sim.fly_valence_dictionary)

In [9]:
for el in sim.odor_scores:
    print(el)
    el -= 10
    print(el)

50.0
40.0
50.0
40.0
50.0
40.0
50.0
40.0
50.0
40.0


In [4]:
decision_interval = 0.01
run_time = sim.simulation_time
num_decision_steps = int(run_time / decision_interval)
logging.info("Number of decision steps: ", num_decision_steps)
physics_steps_per_decision_step = int(decision_interval / sim_params.timestep)
logging.info("Physics steps per decision step: ", physics_steps_per_decision_step)

obs_hist = []
odor_history = []
obs, _ = sim.reset()

#for i in trange(int(run_time / sim.sim_params.timestep)):
for i in trange(num_decision_steps):
    source = sim.choose_odor_exploration()
    sim.update_odor_scores(source)
    print(i, sim.odor_scores)

print(sim.fly_valence_dictionary)

100%|██████████| 1000/1000 [00:00<00:00, 7704.03it/s]

0 [0. 0. 0. 0. 2.]
1 [0. 0. 2. 0. 2.]
2 [0. 0. 2. 2. 2.]
3 [0. 2. 2. 2. 2.]
4 [2. 2. 2. 2. 2.]
5 [2. 2. 2. 2. 4.]
6 [2. 4. 2. 2. 4.]
7 [2. 4. 2. 2. 6.]
8 [2. 4. 2. 4. 6.]
9 [2. 4. 4. 4. 6.]
10 [2. 6. 4. 4. 6.]
11 [4. 6. 4. 4. 6.]
12 [4. 6. 4. 4. 8.]
13 [ 4.  6.  4.  4. 10.]
14 [ 4.  6.  6.  4. 10.]
15 [ 6.  6.  6.  4. 10.]
16 [ 6.  6.  6.  4. 12.]
17 [ 6.  6.  6.  6. 12.]
18 [ 6.  6.  6.  6. 14.]
19 [ 6.  8.  6.  6. 14.]
20 [ 6.  8.  8.  6. 14.]
21 [ 6. 10.  8.  6. 14.]
22 [ 6. 10.  8.  6. 16.]
23 [ 6. 10.  8.  8. 16.]
24 [ 6. 10.  8. 10. 16.]
25 [ 6. 10.  8. 10. 18.]
26 [ 8. 10.  8. 10. 18.]
27 [ 8. 10. 10. 10. 18.]
28 [10. 10. 10. 10. 18.]
29 [10. 10. 10. 12. 18.]
30 [10. 10. 10. 12. 20.]
31 [10. 10. 10. 14. 20.]
32 [12. 10. 10. 14. 20.]
33 [12. 10. 12. 14. 20.]
34 [12. 10. 14. 14. 20.]
35 [12. 12. 14. 14. 20.]
36 [12. 14. 14. 14. 20.]
37 [12. 14. 14. 16. 20.]
38 [12. 14. 14. 18. 20.]
39 [12. 14. 14. 20. 20.]
40 [12. 16. 14. 20. 20.]
41 [12. 18. 14. 20. 20.]
42 [12. 20. 14. 20. 20.]


In [ ]:
# choose a name to save both the video and the trajectory
name = "counting_based_simulation_"
path = Path(f"/home/nmf-learning/flygym-scratch/outputs/plots/{name}")
os.mkdir(path)

In [ ]:
# Plot the trajectory
fly_pos_hist = np.array([obs["fly"][0, :2] for obs in obs_hist])
fig, ax = plt.subplots(1, 1, figsize=(5, 4), tight_layout=True)
ax.scatter(
    [odor_source[0, 0]],
    [odor_source[0, 1]],
    marker="o",
    color="tab:orange",
    s=50,
    label="Attractive",
)
ax.scatter(
    [odor_source[1, 0]],
    [odor_source[1, 1]],
    marker="o",
    color="tab:orange",
    s=50,
)
ax.scatter(
    [odor_source[2, 0]], [odor_source[2, 1]], marker="o", color="tab:orange", s=50
)
ax.scatter(
    [odor_source[3, 0]], [odor_source[3, 1]], marker="o", color="tab:orange", s=50
)
ax.scatter(
    [odor_source[4, 0]], [odor_source[4, 1]], marker="o", color="tab:orange", s=50
)
ax.plot(fly_pos_hist[:, 0], fly_pos_hist[:, 1], color="k", label="Fly trajectory")
ax.set_aspect("equal")
ax.set_xlim(-1, 40)
ax.set_ylim(-10, 10)
ax.set_xlabel("x (mm)")
ax.set_ylabel("y (mm)")
ax.legend(ncols=3, loc="lower center", bbox_to_anchor=(0.5, -0.6))
fig.savefig(
    f"/home/nmf-learning/flygym-scratch/outputs/plots/{name}/odor_taxis_trajectory.png"
)

In [ ]:
# save the video
sim.save_video(
    f"/home/nmf-learning/flygym-scratch/outputs/plots/{name}/odor_taxis_video.mp4"
)